In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
import os

# Data Loading

In [32]:
X_sr = np.load('../data/X_sr.npy')
y_sr = np.load('../data/y_sr.npy')

In [33]:
def make_dataset_dir(path, country, name):
        if os.path.isdir(os.path.join(path, country, name)):
            filelist = [ f for f in os.listdir(os.path.join(path, country, name)) if f.endswith(".npy") ]
            for f in filelist:
                os.remove(os.path.join(os.path.join(path, country, name), f))
            os.rmdir(os.path.join(path, country, name))
            os.mkdir(os.path.join(path, country, name))
        else:
            os.mkdir(os.path.join(path, country, name))

In [7]:
make_dataset_dir('../data', 'serbia', 'data_serbia')

In [34]:
def create_dataset(path, country, name, X, y, filter_out):

    indout = [idx for idx, cl in enumerate(y) if cl in filter_out]

    X = np.array([x for idx, x in enumerate(X) if idx not in indout])
    y = np.array([y for idx, y in enumerate(y) if idx not in indout])
  
    
    
    
    # for s in set(y_sr_cleaned):
    #     print(f"{s} : {len(list(filter(lambda x: x==s, list(y_sr_cleaned))))}")
    np.save( os.path.join(path,country,name,'y.npy'), y)
    for idx, x in enumerate(X):
        np.save(os.path.join(path,country,name,f'{idx}.npy'), x)
    return X, y

In [8]:
X, y = create_dataset('../data', 'serbia', 'data_serbia', X_sr, y_sr, ['not_defined', 'olives'])

In [30]:
_mean = X.mean(axis=0)
_min = X.min(axis=0)
_max = X.max(axis=0)
_len = X.shape[0]

In [31]:
X-_mean

array([[[ 5.88276241e-03,  7.09592522e-03,  1.21294246e-02, ...,
          2.01489704e-02,  1.45921118e-01,  5.71900176e-03],
        [ 1.17280050e-02,  1.27225478e-02,  1.82442349e-02, ...,
          3.24794918e-02,  1.37379666e-01,  1.02349458e-02],
        [-5.56855509e-03, -1.37597107e-03,  2.04436577e-04, ...,
          5.47383632e-02,  1.33025246e-01, -6.27011551e-02],
        ...,
        [-1.35654252e-02, -1.45089999e-02, -2.14076125e-02, ...,
          7.30200820e-02,  2.35339478e-01, -1.41642358e-01],
        [-1.49669812e-02, -1.51074785e-02, -1.97823744e-02, ...,
          6.66024832e-02,  2.05308021e-01, -1.37896727e-01],
        [-1.87320494e-02, -1.80166778e-02, -2.05614757e-02, ...,
          6.32202514e-02,  1.88875063e-01, -1.37355418e-01]],

       [[ 7.79276241e-03,  8.88842522e-03,  1.72344246e-02, ...,
         -3.01210686e-02, -5.99993256e-02,  4.80987112e-02],
        [ 3.68371925e-03,  3.41969070e-03,  9.73994917e-03, ...,
         -1.18393463e-02, -6.16009122e

## Stratified splits for Vojvodina dataset

In [35]:
def prepare_datasets(path, country, name, Xpath, ypath, filter_out):
    X = np.load(Xpath)
    y = np.load(ypath)
    make_dataset_dir(path, country, name)
    X, y = create_dataset(path, country, name,X,y,filter_out)
    _mean = X.mean(axis=0)
    _std = X.std(axis=0)
    _min = X.min(axis=0)
    _max = X.max(axis=0)
    _len = X.shape[0]
    np.save( os.path.join(path,country,'mean.npy'), _mean)
    np.save( os.path.join(path,country,'std.npy'), _std)
    np.save( os.path.join(path,country,'min.npy'), _min)
    np.save( os.path.join(path,country,'max.npy'), _max)
    np.save( os.path.join(path,country,'len.npy'), _len)
    
    
    make_dataset_dir(path, country, name)
    X, y = create_dataset(path, country, name,(X - _mean)/(_len*_std),y,[])
    
    
    X_0102, X_0304, y_0102, y_0304 = train_test_split(X, y, test_size=0.5, random_state=1, shuffle=True, stratify=y)
    X_01, X_02, y_01, y_02 = train_test_split(X_0102, y_0102, test_size=0.5, random_state=1, shuffle=True, stratify=y_0102)
    X_03, X_04, y_03, y_04 = train_test_split(X_0304, y_0304, test_size=0.5, random_state=1, shuffle=True, stratify=y_0304)
    
    make_dataset_dir(path, country, f'{name}_01')
    _, _ = create_dataset(path, country, f'{name}_01', (X_01 - _mean)/(_len*_std),y_01,[])

    make_dataset_dir(path, country, f'{name}_02')
    _, _ = create_dataset(path, country, f'{name}_02', (X_02 - _mean)/(_len*_std),y_02,[])

    make_dataset_dir(path, country, f'{name}_03')
    _, _ = create_dataset(path, country, f'{name}_03', (X_03 - _mean)/(_len*_std),y_03,[])

    make_dataset_dir(path, country, f'{name}_04')
    _, _ = create_dataset(path, country, f'{name}_04', (X_04 - _mean)/(_len*_std),y_04,[])
        
    
    

In [36]:
prepare_datasets('../data', 'serbia', 'data_serbia', '../data/X_sr.npy', '../data/y_sr.npy', ['not_defined', 'olives'])

In [48]:
for s in set(y):
    print(f"{s} : {len(list(filter(lambda x: x==s, list(y))))}")

sugarbeet : 14
maize : 2108
rapeseed : 63
barley : 187
soya : 234
sunflower : 1033
wheat : 763


In [49]:
y

array(['soya', 'maize', 'wheat', ..., 'maize', 'maize', 'maize'],
      dtype='<U9')

In [30]:
indout = [idx for idx, cl in enumerate(ssl) if cl in ['not_defined', 'olives', 'sugarbeet']]
X_sr_cleaned = np.array([x for idx, x in enumerate(X_sr) if idx not in indout])
y_sr_cleaned = np.array([y for idx, y in enumerate(y_sr) if idx not in indout])

In [2]:
from grfcrops.datasets.vojvodina import VojvodinaDataset
dataset = VojvodinaDataset(country='serbia', name='data_serbia_01')

ModuleNotFoundError: No module named 'datasets'